## Evaluation of models which contains english generation

In [47]:
import pandas as pd
from tqdm import tqdm
import fasttext
from huggingface_hub import hf_hub_download

# Télécharger et charger le modèle GlotLID 
model_path = hf_hub_download(repo_id="cis-lmu/glotlid", filename="model.bin")
model = fasttext.load_model(model_path)

# Fonction de détection de l'anglais avec GlotLID
def is_english(text, threshold=0.8):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return False
    try:
        text = text.replace("\n", " ").strip()  # remove newlines because of a error 
        label, prob = model.predict(text)
        lang_code = label[0].replace("__label__", "")
        return lang_code.startswith("eng") and prob[0] >= threshold
    except Exception as e:
        print(f"Error processing text: {text[:50]}... -> {e}")
        return False

# analyze
def analyze_english_percentage(setting):
    modeles = [
        "bloom-560m", "bloom-3b", "gpt2-fr", "xglm-2", "bloom-7b",
        "vigogne-2-7b", "croissantbase", "croissant-it",
        "llama-3.2-3b-it", "llama-3.2-3b",
        "gemma-2-2b", "gemma-2-2b-it",
        "mistral-7b-instruct-v0.3", "mistral-7b-v0.3"
    ]
    
    results = []

    for modele in modeles:
        print(f"Traitement du modèle : {modele}")
        csv_file = f"../annotated_texts/FR/{setting}/annotated-coverletter_{setting}_fr_{modele}.csv"
        df = pd.read_csv(csv_file)

        column_name = "output"

        tqdm.pandas(desc=f"Analyse des phrases pour {modele}")
        df["is_english"] = df[column_name].progress_apply(lambda x: is_english(x))

        english_count = df["is_english"].sum()
        total_count = len(df)
        percentage_english = (english_count / total_count) * 100

        results.append({
            "Rank": None,
            "Model": modele,
            "English_percentage": round(percentage_english, 2)
        })

    results_df = pd.DataFrame(results)
    results_df["Rank"] = results_df["English_percentage"].rank(ascending=False, method="first").astype(int)
    results_df = results_df.sort_values(by="English_percentage", ascending=False).reset_index(drop=True)  
    results_df["English_percentage"] = results_df["English_percentage"].map("{:.2f}".format)
    print(results_df)

    #output_file = f"./results_percentage_english_{setting}.csv"
    #results_df.to_csv(output_file, index=False)
    #print(f"Résultats sauvegardés dans {output_file}")

    return results_df


In [48]:
res_neutral = analyze_english_percentage("neutral")

Traitement du modèle : bloom-560m


Analyse des phrases pour bloom-560m: 100%|██████████| 4992/4992 [00:03<00:00, 1468.08it/s]


Traitement du modèle : bloom-3b


Analyse des phrases pour bloom-3b: 100%|██████████| 4992/4992 [00:03<00:00, 1265.82it/s]


Traitement du modèle : gpt2-fr


...

Traitement du modèle : mistral-7b-instruct-v0.3


Analyse des phrases pour mistral-7b-instruct-v0.3: 100%|██████████| 4992/4992 [00:03<00:00, 1353.12it/s]


Traitement du modèle : mistral-7b-v0.3


Analyse des phrases pour mistral-7b-v0.3: 100%|██████████| 4992/4992 [00:03<00:00, 1400.35it/s]

    Rank                     Model English_percentage
0      1           mistral-7b-v0.3              40.16
1      2  mistral-7b-instruct-v0.3               3.65
2      3             gemma-2-2b-it               1.11
3      4              croissant-it               0.14
4      5           llama-3.2-3b-it               0.08
5      6              llama-3.2-3b               0.08
6      7              vigogne-2-7b               0.06
7      8             croissantbase               0.06
8      9                   gpt2-fr               0.04
9     10                  bloom-7b               0.02
10    11                gemma-2-2b               0.02
11    12                bloom-560m               0.00
12    13                  bloom-3b               0.00
13    14                    xglm-2               0.00


In [49]:
res_gendered= analyze_english_percentage("gendered")

Traitement du modèle : bloom-560m


Analyse des phrases pour bloom-560m: 100%|██████████| 4968/4968 [00:03<00:00, 1346.04it/s]


Traitement du modèle : bloom-3b


Analyse des phrases pour bloom-3b: 100%|██████████| 4968/4968 [00:03<00:00, 1304.42it/s]


Traitement du modèle : gpt2-fr


...

Traitement du modèle : mistral-7b-instruct-v0.3


Analyse des phrases pour mistral-7b-instruct-v0.3: 100%|██████████| 4968/4968 [00:03<00:00, 1380.02it/s]


Traitement du modèle : mistral-7b-v0.3


Analyse des phrases pour mistral-7b-v0.3: 100%|██████████| 4968/4968 [00:03<00:00, 1425.49it/s]

    Rank                     Model English_percentage
0      1           mistral-7b-v0.3              48.71
1      2  mistral-7b-instruct-v0.3               8.98
2      3             gemma-2-2b-it               8.64
3      4              vigogne-2-7b               0.34
4      5             croissantbase               0.14
5      6                gemma-2-2b               0.12
6      7                   gpt2-fr               0.04
7      8           llama-3.2-3b-it               0.04
8      9              llama-3.2-3b               0.04
9     10                bloom-560m               0.00
10    11                  bloom-3b               0.00
11    12                    xglm-2               0.00
12    13                  bloom-7b               0.00
13    14              croissant-it               0.00


In [51]:
res_gendered.to_latex(index =False)

'\\begin{tabular}{rll}\n\\toprule\nRank & Model & English_percentage \\\\\n\\midrule\n1 & mistral-7b-v0.3 & 48.71 \\\\\n2 & mistral-7b-instruct-v0.3 & 8.98 \\\\\n3 & gemma-2-2b-it & 8.64 \\\\\n4 & vigogne-2-7b & 0.34 \\\\\n5 & croissantbase & 0.14 \\\\\n6 & gemma-2-2b & 0.12 \\\\\n7 & gpt2-fr & 0.04 \\\\\n8 & llama-3.2-3b-it & 0.04 \\\\\n9 & llama-3.2-3b & 0.04 \\\\\n10 & bloom-560m & 0.00 \\\\\n11 & bloom-3b & 0.00 \\\\\n12 & xglm-2 & 0.00 \\\\\n13 & bloom-7b & 0.00 \\\\\n14 & croissant-it & 0.00 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [52]:
res_neutral.to_latex(index =False)

'\\begin{tabular}{rll}\n\\toprule\nRank & Model & English_percentage \\\\\n\\midrule\n1 & mistral-7b-v0.3 & 40.16 \\\\\n2 & mistral-7b-instruct-v0.3 & 3.65 \\\\\n3 & gemma-2-2b-it & 1.11 \\\\\n4 & croissant-it & 0.14 \\\\\n5 & llama-3.2-3b-it & 0.08 \\\\\n6 & llama-3.2-3b & 0.08 \\\\\n7 & vigogne-2-7b & 0.06 \\\\\n8 & croissantbase & 0.06 \\\\\n9 & gpt2-fr & 0.04 \\\\\n10 & bloom-7b & 0.02 \\\\\n11 & gemma-2-2b & 0.02 \\\\\n12 & bloom-560m & 0.00 \\\\\n13 & bloom-3b & 0.00 \\\\\n14 & xglm-2 & 0.00 \\\\\n\\bottomrule\n\\end{tabular}\n'